In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from env import get_db_url
import pandas as pd

import acquire
import prepare

In [6]:
df = pd.read_sql('SELECT * FROM country', get_db_url('world'))
df.head()


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL


In [11]:
df.groupby('Region').Name.count()

Region
Antarctica                    5
Australia and New Zealand     5
Baltic Countries              3
British Islands               2
Caribbean                    24
Central Africa                9
Central America               8
Eastern Africa               20
Eastern Asia                  8
Eastern Europe               10
Melanesia                     5
Micronesia                    7
Micronesia/Caribbean          1
Middle East                  18
Nordic Countries              7
North America                 5
Northern Africa               7
Polynesia                    10
South America                14
Southeast Asia               11
Southern Africa               5
Southern Europe              15
Southern and Central Asia    14
Western Africa               17
Western Europe                9
Name: Name, dtype: int64

In [16]:
world = sns.load_dataset('World')
sns.histplot(world.Region)
plt.show

ValueError: 'World' is not one of the example datasets.

October 25 warmup: Warmup - Your boss rushes in and says "QUICK! I need you to prep our data to get ready for a MVP classification problem."


In [8]:
df = sns.load_dataset('tips')

In [9]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [19]:
def prep_tips_data(df):
    cols_to_drop = ['sex', 'smoker', 'day', 'time']
    df = df.drop(columns = cols_to_drop)
    
    dummy_df = pd.get_dummies(df[['sex', 'smoker', 'day', 'time']], dummy_na = False, drop_first=True)
    df = pd.concat([df, dummy_df], axis =1)
    
    return df

In [20]:
prep_tips_data(df)

KeyError: "None of [Index(['sex', 'smoker', 'day', 'time'], dtype='object')] are in the [columns]"

- how andrew did it:

In [21]:
#now to get rid of categorical variables:
pd.get_dummies(df[['sex', 'smoker', 'day', 'time']])

,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,0,1,0,1,0,0,0,1,0,1
1,1,0,0,1,0,0,0,1,0,1
2,1,0,0,1,0,0,0,1,0,1
3,1,0,0,1,0,0,0,1,0,1
4,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
239,1,0,0,1,0,0,1,0,0,1
240,0,1,1,0,0,0,1,0,0,1
241,1,0,1,0,0,0,1,0,0,1
242,1,0,0,1,0,0,1,0,0,1


In [22]:
#now we save that back to a dataframe and concat:
df_dum = pd.get_dummies(df[['sex', 'smoker', 'day', 'time']])

In [24]:
# concat
df = pd.concat ([df, df_dum], axis =1)

In [25]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,Female,No,Sun,Dinner,2,0,1,0,1,0,0,0,1,0,1
1,10.34,1.66,Male,No,Sun,Dinner,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3.50,Male,No,Sun,Dinner,3,1,0,0,1,0,0,0,1,0,1
3,23.68,3.31,Male,No,Sun,Dinner,2,1,0,0,1,0,0,0,1,0,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0,1,0,1,0,0,0,1,0,1


# Jan 10 2023

In [ ]:
df = pd.read_sql('SELECT * FROM country', get_db_url('world'))
df.head()

# Jan 19
Write a program that returns a list that contains only the elements that are common between two lists (without duplicates). Make sure your program works on two lists of different sizes.


In [11]:
A = [1,2,3,3,3,4,5]
B = [6,7,8,9,3,2,1]


In [27]:
def common(A, B):
    common = []
    for x in A:
        if x in B:
            if x not in common:    
                common.append(x)
    return (common)

In [23]:
def common(A, B):
    common = []
    for x in A:
        if x in B:
            common.append(x)
    return list(set(common))

In [28]:
common(A, B)

[1, 2, 3]

In [22]:
common(B,A)

[3, 2, 1]

In [25]:
def count_common(A,B):
    common = 0
    for x in A:
        if x in B:
            common+=1
    return common

In [26]:
count_common(A,B)

5

# Jan 20
Write a function that takes a list of numbers and returns a list with two elements:
The first element should be the sum of all even numbers in the list.
The second element should be the sum of all odd numbers in the list.
sum_odd_and_even([1, 2, 3, 4, 5, 6]) ➞ [12, 9]

In [1]:
A = [1,2,3,4,5,6]

In [7]:
evens =
             if x % 2 ==0
odds = for x in A:
        if x % 2 == 1

SyntaxError: invalid syntax (1015147289.py, line 1)

In [ ]:
def sum_odd_and_even(nums):
    products = [evens, odds]
    evens = 0
    odds = 0
    